In [1]:
import pandas as pd
import datetime
import pymmwr
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import sqlite3
from collections import defaultdict
from scipy import stats
import seaborn as sns
%matplotlib inline

vac_cov_seasonal = pd.read_csv('../data/vac_coverage_by_birth_year_seasonal_2010.csv')
vac_cov_mono = pd.read_csv('../data/monovalent_spline_coverage_by_birth_year.csv')


pandemic_file = '../raw_data/Historic_Flu_Pandemics.csv'
Thompson_data = '../raw_data/Thompson_flu_counts.csv'
cumulative_incidence_file = '../data/weekly_incidence_simplified.csv'
MESA_subtype_file = '../raw_data/subtype_fractions_by_season.csv'
intensity_file = '../data/Intensity_and_Frequency.csv'
vac_prob = pd.read_csv('../raw_data/Marshfield_vaccination_timing.csv')
mesa_prob = pd.read_csv('../raw_data/Marshfield_infection_timing.csv')

def weekly_demo_function(year,
                         week, 
                         birth_year,
                         waning_time_days=180):
    
    '''
    Given a birth year, a year, and an mmwr week in that year, this returns
    the fraction of the population born in that birth year that experiences that
    partciular week.
    '''
    
    # Converts the waning period into a datetime timedelta object
    waning_period = datetime.timedelta(waning_time_days)
    
    # Calculates the effective first day that people in a particular birth year are susceptible and the
    # last day that someone born in that year wanes
    initial_day = pymmwr.date_to_epiweek(datetime.date(birth_year, 1, 1) 
                                         + waning_period)
    final_day = pymmwr.date_to_epiweek(datetime.date(birth_year, 12, 31) 
                                         + waning_period)
    
    birth_year_max_week = get_max_week(birth_year)
   
    # Turns those days into weeks
    initial_week = (initial_day.year, initial_day.week)
    final_week = (final_day.year, final_day.week)
    
    # If the week in question is before maternal waning has occurred then no individual in that
    # birth year class is "exposed"
    if (year, week) < initial_week:
        demo_frac = 0
    
    # If we're past the last day of waning, then everyone is exposed
    elif (year, week) >= final_week:
        demo_frac = 1
    
    # Otherwise, we need to calculate a fraction of the population exposed
    else:
        if year == birth_year:
            multiplier = int(week) - int(initial_week[1]) + 1
        elif year == birth_year + 1:
            multiplier = (int(birth_year_max_week) - int(initial_week[1]) + 1) + int(week)
        demo_frac = (multiplier / birth_year_max_week)

    return demo_frac


def season_float_to_label(season):
    if season==2009.5:
        label = '2009Pan'
    else:
        label = str(int(season)-1) + '-' + str(int(season))
    return label

def get_max_week(year):
    '''
    Given a year, gets the maximum MMWR week for that year (either 52 or 53)
    '''
    birth_year_max_week = 0
    for day in range(1, 32):
        week = pymmwr.date_to_epiweek(datetime.date(year, 12, day)).week
        if week > birth_year_max_week:
            birth_year_max_week = week
    return birth_year_max_week


waning_time_days = 180
childhood_end_years = 12
season_start = 40
pandemic_start = 15
pandemic_end = 47
min_birth_year = 1918
max_birth_year = 2018



MESA_subtype_fracs = pd.read_csv(MESA_subtype_file, index_col=0)
weekly_iav_incidence = pd.read_csv(cumulative_incidence_file, index_col=0)
weekly_iav_incidence = weekly_iav_incidence.fillna(0)

historical_frac_a = pd.read_csv(Thompson_data)
historical_frac_a.index = [float(s.split('-')[-1]) for s in historical_frac_a.Season]

historical_pandemics = pd.read_csv(pandemic_file, index_col=0)
Intensity_and_Frequency = pd.read_csv(intensity_file, index_col=0)


In [2]:
# Process vaccination probabilities for 2009Pan and 2010 Seasons
monovac = vac_prob[vac_prob['Vaccine type'] == '2009Pan'].copy()
monovac.columns = ['Vaccine type', 'Year', 'Week', 'Count', 'PMF_mono']
seasonvac = vac_prob[vac_prob['Vaccine type'] == '2010'].copy()
seasonvac.columns = ['Vaccine type', 'Year', 'Week', 'Count', 'PMF_season']
vac_prob = monovac.merge(seasonvac, how='outer', on=['Year', 'Week'])[['Year', 'Week','PMF_mono', 'PMF_season']].fillna(0).sort_values(['Year', 'Week'])

In [3]:
# Process coverage
vac_cov_mono = vac_cov_mono[['Birth_year', 'Coverage']]
vac_cov_mono.columns = ['Birth_year', '2009_mono']
vac_cov_seasonal = vac_cov_seasonal[vac_cov_seasonal.Season==2010]
vac_cov_seasonal.columns = ['Birth_year', 'Season', '2009_seasonal']
vac_cov = vac_cov_seasonal[['Birth_year', '2009_seasonal']].merge(vac_cov_mono[['Birth_year', '2009_mono']])
vac_cov.index = vac_cov.Birth_year

In [4]:
new_rows = []


# Calculate unscaled weekly attack probabilities 
attack_prob = 0.28
attack_rate = -np.log(1 - attack_prob)

subplot = 1

weekly_iav_incidence['Week'] = weekly_iav_incidence.WEEK
weekly_iav_incidence['Year'] = weekly_iav_incidence.index

for season, season_df in weekly_iav_incidence.groupby('SEASON'):
        #plt.subplot(3,4,subplot)
        #subplot += 1
    if season == 2009.5:
        
        # For the pandemic, we're using the infection PMF from Marshfield
        # since the pandemic season timing in Marshfield seems markedly different
        # from national data
        seasonstr = '2009Pan'
        infect_col = 'PMF_A_Mesa'
        
        mesa = mesa_prob[mesa_prob['Season'] == seasonstr].copy()
        tempdf = season_df.copy()
        tempdf = tempdf.merge(mesa, how='outer', on=['Year', 'Week']).fillna(0)
        # Merge in vaccination PMFs
        tempdf = tempdf.merge(vac_prob, how='outer', on=['Year', 'Week']).fillna(0)
        tempdf = tempdf.sort_values(['Year', 'Week'])
        for birth_year in range(1918, 2010):
            plotdf = tempdf.copy()
            cov = vac_cov.loc[birth_year, ['2009_seasonal', '2009_mono']]
            season_vac_rates = -np.log(1 - cov)

            for index, row in plotdf.iterrows():
                plotdf.loc[index, 'frac_experienced'] = weekly_demo_function(row.Year, row.Week, birth_year)
            
            # Get the attack rate for each week of the season and calculate the per-week infection probabilities
            weekly_attack_rate = attack_rate * Intensity_and_Frequency.loc[season, 'Intensity'] * plotdf[infect_col] * plotdf['frac_experienced']
            raw_infection_probs = 1 - np.exp(-weekly_attack_rate)
            
            # Calculate the probability of no infection prior to each week
            prob_no_infection = np.array(1-raw_infection_probs).cumprod()
            prob_no_infection = [1] + list(prob_no_infection)[0:-1]
        
            weekly_vac_rates_mono = season_vac_rates['2009_mono'] * plotdf['PMF_mono'] * plotdf['frac_experienced']
            weekly_vac_rates_season = season_vac_rates['2009_seasonal'] * plotdf['PMF_season'] * plotdf['frac_experienced']

            raw_vac_probs_mono = 1 - np.exp(-weekly_vac_rates_mono)
            raw_vac_probs_seasonal = 1 - np.exp(-weekly_vac_rates_season)
            
            prob_no_mono_vac = np.array(1-raw_vac_probs_mono).cumprod()
            prob_no_mono_vac = [1] + list(prob_no_mono_vac)[0:-1]
            
            prob_no_seasonal_vac = np.array(1-raw_vac_probs_seasonal).cumprod()
            prob_no_seasonal_vac = [1] + list(prob_no_seasonal_vac)[0:-1]
            
            plotdf['Mono_vac'] = raw_vac_probs_mono * np.array(prob_no_mono_vac)
            plotdf['Seasonal_vac'] = raw_vac_probs_seasonal * np.array(prob_no_seasonal_vac)
            plotdf['Infection'] = raw_infection_probs * np.array(prob_no_infection)
            
            # Probability that all three happen
            plotdf['Mono_seasonal_inf'] = plotdf.Mono_vac * plotdf.Seasonal_vac * plotdf.Infection
            
            # Probability that you get both vaccines but are not infected
            plotdf['Mono_seasonal'] = plotdf.Mono_vac * plotdf.Seasonal_vac - plotdf.Mono_seasonal_inf
            
            # Probability that you get monovalent and are infected but not seasonal
            
            plotdf['Mono_inf'] = plotdf.Mono_vac * plotdf.Infection - plotdf.Mono_seasonal_inf
            
            # Probability that you get Seasonal vaccine and are infected but don't get the monovalent vaccine
            plotdf['Seasonal_inf'] = plotdf.Seasonal_vac * plotdf.Infection - plotdf.Mono_seasonal_inf
            
            plotdf['unexposed'] = plotdf.Mono_vac + plotdf.Seasonal_vac + plotdf.Infection
            plotdf['unexposed'] = plotdf.unexposed - plotdf.Mono_seasonal_inf - plotdf.Mono_seasonal - plotdf.Mono_inf - plotdf.Seasonal_inf
            plotdf['unexposed'] = (1 - plotdf.unexposed).cumprod()
            plotdf['unexposed'] = [1] + list(plotdf.unexposed)[0:-1]
            
            # Seasonal / season + mono + inf
            plotdf['Season_to_season+mono+inf'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Seasonal / seasonal + mono
            plotdf['Season_to_season+mono'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac)

            # Seasonal / seasonal + inf
            plotdf['Season_to_season+inf'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Infection)

            # Mono / season + mono + inf
            plotdf['Mono_to_season+mono+inf'] = plotdf.Mono_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Mono / seasonal + mono
            plotdf['Mono_to_season+mono'] = plotdf.Mono_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac)

            # Mono / mono + inf
            plotdf['Mono_to_mono+inf'] = plotdf.Mono_vac / (plotdf.Mono_vac + plotdf.Infection)
            
            # Inf / season + mono + inf
            plotdf['Inf_to_season+mono+inf'] = plotdf.Infection / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Inf / seasonal + inf
            plotdf['Inf_to_season+inf'] = plotdf.Infection / (plotdf.Seasonal_vac + plotdf.Infection)

            # Inf / mono + inf
            plotdf['Inf_to_mono+inf'] = plotdf.Infection / (plotdf.Mono_vac + plotdf.Infection)
            
            
            # Get first exposures
            plotdf['Inf_first'] = plotdf.Infection - (plotdf.Mono_seasonal_inf + plotdf.Mono_inf + plotdf.Seasonal_inf)
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Mono_seasonal_inf * plotdf['Inf_to_season+mono+inf']
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Mono_inf * plotdf['Inf_to_mono+inf']
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Seasonal_inf * plotdf['Inf_to_season+inf']
            plotdf['Inf_first'] = plotdf.Inf_first * plotdf.unexposed
            
            plotdf['Mono_first'] = plotdf.Mono_vac - (plotdf.Mono_seasonal_inf + plotdf.Mono_inf + plotdf.Mono_seasonal)
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_seasonal_inf * plotdf['Mono_to_season+mono+inf']
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_inf * plotdf['Mono_to_mono+inf']
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_seasonal * plotdf['Mono_to_season+mono']
            plotdf['Mono_first'] = plotdf.Mono_first * plotdf.unexposed

            plotdf['Season_first'] = plotdf.Seasonal_vac - (plotdf.Mono_seasonal_inf + plotdf.Seasonal_inf + plotdf.Mono_seasonal)
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Mono_seasonal_inf * plotdf['Season_to_season+mono+inf']
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Seasonal_inf * plotdf['Season_to_season+inf']
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Mono_seasonal * plotdf['Season_to_season+mono']
            plotdf['Season_first'] = plotdf.Season_first * plotdf.unexposed
            
            plotdf['Ordinal week'] = range(0, len(plotdf))
            plotdf = plotdf.fillna(0)
            plotdf = plotdf[(plotdf.Week < 47) & (plotdf.Year == 2009)]
            #plotdf.plot(x='Ordinal week', y=['Season_first', 'Mono_first', 'Inf_first', 'unexposed'], ax =plt.gca(), marker='o')

            season_probs = plotdf.sum()[['Seasonal_vac', 'Mono_vac', 'Season_first', 'Mono_first', 'Inf_first']]
            new_rows.append([birth_year, 2009.5] + list(season_probs))
            #plt.title(seasonstr)


In [5]:

for season, season_df in weekly_iav_incidence.groupby('SEASON'):
        #plt.subplot(3,4,subplot)
        #subplot += 1
    if season == 2010:
        
        # For the pandemic, we're using the infection PMF from Marshfield
        # since the pandemic season timing in Marshfield seems markedly different
        # from national data
        seasonstr = '2010'
        infect_col = 'PMF_A_Mesa'
        
        mesa = mesa_prob[mesa_prob['Season'] == seasonstr].copy()
        tempdf = season_df.copy()
        tempdf = tempdf.merge(mesa, how='outer', on=['Year', 'Week']).fillna(0)
        # Merge in vaccination PMFs
        tempdf = tempdf.merge(vac_prob, how='outer', on=['Year', 'Week']).fillna(0)
        tempdf = tempdf.sort_values(['Year', 'Week'])
        for birth_year in range(1918, 2010):
            plotdf = tempdf.copy()
            cov = vac_cov.loc[birth_year, ['2009_seasonal', '2009_mono']]
            season_vac_rates = -np.log(1 - cov)

            for index, row in plotdf.iterrows():
                plotdf.loc[index, 'frac_experienced'] = weekly_demo_function(row.Year, row.Week, birth_year)
            
            # Get the attack rate for each week of the season and calculate the per-week infection probabilities
            weekly_attack_rate = attack_rate * Intensity_and_Frequency.loc[season, 'Intensity'] * plotdf[infect_col] * plotdf['frac_experienced']
            raw_infection_probs = 1 - np.exp(-weekly_attack_rate)
            
            # Calculate the probability of no infection prior to each week
            prob_no_infection = np.array(1-raw_infection_probs).cumprod()
            prob_no_infection = [1] + list(prob_no_infection)[0:-1]
        
            weekly_vac_rates_mono = season_vac_rates['2009_mono'] * plotdf['PMF_mono'] * plotdf['frac_experienced']
            weekly_vac_rates_season = season_vac_rates['2009_seasonal'] * plotdf['PMF_season'] * plotdf['frac_experienced']

            raw_vac_probs_mono = 1 - np.exp(-weekly_vac_rates_mono)
            raw_vac_probs_seasonal = 1 - np.exp(-weekly_vac_rates_season)
            
            prob_no_mono_vac = np.array(1-raw_vac_probs_mono).cumprod()
            prob_no_mono_vac = [1] + list(prob_no_mono_vac)[0:-1]
            
            prob_no_seasonal_vac = np.array(1-raw_vac_probs_seasonal).cumprod()
            prob_no_seasonal_vac = [1] + list(prob_no_seasonal_vac)[0:-1]
            
            plotdf['Mono_vac'] = raw_vac_probs_mono * np.array(prob_no_mono_vac)
            plotdf['Seasonal_vac'] = raw_vac_probs_seasonal * np.array(prob_no_seasonal_vac)
            plotdf['Infection'] = raw_infection_probs * np.array(prob_no_infection)
            
            
            # Probability that all three happen
            plotdf['Mono_seasonal_inf'] = plotdf.Mono_vac * plotdf.Seasonal_vac * plotdf.Infection
            
            # Probability that you get both vaccines but are not infected
            plotdf['Mono_seasonal'] = plotdf.Mono_vac * plotdf.Seasonal_vac - plotdf.Mono_seasonal_inf
            
            # Probability that you get monovalent and are infected but not seasonal
            
            plotdf['Mono_inf'] = plotdf.Mono_vac * plotdf.Infection - plotdf.Mono_seasonal_inf
            
            # Probability that you get Seasonal vaccine and are infected but don't get the monovalent vaccine
            plotdf['Seasonal_inf'] = plotdf.Seasonal_vac * plotdf.Infection - plotdf.Mono_seasonal_inf
            
            plotdf['unexposed'] = plotdf.Mono_vac + plotdf.Seasonal_vac + plotdf.Infection
            plotdf['unexposed'] = plotdf.unexposed - plotdf.Mono_seasonal_inf - plotdf.Mono_seasonal - plotdf.Mono_inf - plotdf.Seasonal_inf
            plotdf['unexposed'] = (1 - plotdf.unexposed).cumprod()
            plotdf['unexposed'] = [1] + list(plotdf.unexposed)[0:-1]
            
            # Seasonal / season + mono + inf
            plotdf['Season_to_season+mono+inf'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Seasonal / seasonal + mono
            plotdf['Season_to_season+mono'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac)

            # Seasonal / seasonal + inf
            plotdf['Season_to_season+inf'] = plotdf.Seasonal_vac / (plotdf.Seasonal_vac + plotdf.Infection)

            # Mono / season + mono + inf
            plotdf['Mono_to_season+mono+inf'] = plotdf.Mono_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Mono / seasonal + mono
            plotdf['Mono_to_season+mono'] = plotdf.Mono_vac / (plotdf.Seasonal_vac + plotdf.Mono_vac)

            # Mono / mono + inf
            plotdf['Mono_to_mono+inf'] = plotdf.Mono_vac / (plotdf.Mono_vac + plotdf.Infection)
            
            # Inf / season + mono + inf
            plotdf['Inf_to_season+mono+inf'] = plotdf.Infection / (plotdf.Seasonal_vac + plotdf.Mono_vac + plotdf.Infection)
            
            # Inf / seasonal + inf
            plotdf['Inf_to_season+inf'] = plotdf.Infection / (plotdf.Seasonal_vac + plotdf.Infection)

            # Inf / mono + inf
            plotdf['Inf_to_mono+inf'] = plotdf.Infection / (plotdf.Mono_vac + plotdf.Infection)
            
            
            # Get first exposures
            plotdf['Inf_first'] = plotdf.Infection - (plotdf.Mono_seasonal_inf + plotdf.Mono_inf + plotdf.Seasonal_inf)
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Mono_seasonal_inf * plotdf['Inf_to_season+mono+inf']
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Mono_inf * plotdf['Inf_to_mono+inf']
            plotdf['Inf_first'] = plotdf.Inf_first + plotdf.Seasonal_inf * plotdf['Inf_to_season+inf']
            plotdf['Inf_first'] = plotdf.Inf_first * plotdf.unexposed
            
            plotdf['Mono_first'] = plotdf.Mono_vac - (plotdf.Mono_seasonal_inf + plotdf.Mono_inf + plotdf.Mono_seasonal)
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_seasonal_inf * plotdf['Mono_to_season+mono+inf']
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_inf * plotdf['Mono_to_mono+inf']
            plotdf['Mono_first'] = plotdf.Mono_first + plotdf.Mono_seasonal * plotdf['Mono_to_season+mono']
            plotdf['Mono_first'] = plotdf.Mono_first * plotdf.unexposed

            plotdf['Season_first'] = plotdf.Seasonal_vac - (plotdf.Mono_seasonal_inf + plotdf.Seasonal_inf + plotdf.Mono_seasonal)
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Mono_seasonal_inf * plotdf['Season_to_season+mono+inf']
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Seasonal_inf * plotdf['Season_to_season+inf']
            plotdf['Season_first'] = plotdf.Season_first + plotdf.Mono_seasonal * plotdf['Season_to_season+mono']
            plotdf['Season_first'] = plotdf.Season_first * plotdf.unexposed
            
            plotdf['Ordinal week'] = range(0, len(plotdf))
            plotdf = plotdf.fillna(0)
            plotdf = plotdf[((plotdf.Week >= 47) & (plotdf.Year == 2009)) |
                            (plotdf.Year == 2010)]
            #plotdf.plot(x='Ordinal week', y=['Season_first', 'Mono_first', 'Inf_first'], ax =plt.gca(), marker='o')

            season_probs = plotdf.sum()[['Seasonal_vac', 'Mono_vac', 'Season_first', 'Mono_first', 'Inf_first']]
            new_rows.append([birth_year, 2010] + list(season_probs))
            #plt.title(seasonstr)


In [6]:
pandemic_seasons = pd.DataFrame(new_rows, columns=['Birth_year', 'Season', 'seasonal_coverage', 'mono_coverage', 'seasonal_vaccine', 'monovalent_vaccine', 'infection'])
pandemic_seasons.to_csv('../data/Marshfield_adjusted_exposure_probs.csv', index=False)